In [1]:
import os
import copy
import time
import random
from tqdm import tqdm

import PIL
from PIL import Image


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torchvision import datasets, transforms, models

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score
# fix seeds
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 2019
seed_everything(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.enabled = False
print(device)

cuda


In [31]:

df_class = pd.read_csv('../data/class.csv')
df_train = pd.read_csv('../data/train.csv')
df_train = df_train[['img_file', 'class']]
df_train.replace(196, 0, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(df_train['img_file'], df_train['class'], stratify=df_train['class'], test_size=0.2, random_state=SEED)

X_train = X_train.values
X_val = X_val.values
y_train = y_train.values
y_val = y_val.values

TRAIN_DATA_PATH = '../data/train_crop/'
TEST_DATA_PATH = '../data/test_crop/'

class TrainImages(Dataset):
    def __init__(self, images, labels, mode=None, transforms=None):
        self.images = images
        self.labels = labels
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __len__(self):
        return self.images.shape[0]
        
    def __getitem__(self, idx):
        image = Image.open(TRAIN_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        label = self.labels[idx]
        
        return image, label
    
    
class TestImages(Dataset):
    def __init__(self, images, mode=None, transforms=None):
        self.images = images
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __len__(self):
        return self.images.shape[0]
        
    def __getitem__(self, idx):
        image = Image.open(TEST_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        
        return image

transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ])
}

batch_size = 64

train_dataset = TrainImages(images=X_train, labels=y_train, mode='train', transforms=transform)
val_dataset = TrainImages(images=X_val, labels=y_val, mode='val', transforms=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader
}

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, device, PATH, epochs=20):
    start = time.time()

    num_classes = 196

    best_model_weights = copy.deepcopy(model.state_dict())
    best_f1 = 0.0

    for epoch in tqdm(range(epochs)):
        print("EPOCH {} / {}: ".format(epoch+1, epochs))
        print("-" * 10)

        epoch_loss = 0.0
        phase = 'train'

        for batch_index, (batch_inputs, batch_labels) in enumerate(dataloaders[phase]):
            batch_inputs = batch_inputs.cuda()
            batch_labels = batch_labels.cuda()

            optimizer.zero_grad()
            outputs = model(batch_inputs)
#             _, preds = torch.max(outputs, 1)
            batch_loss = criterion(outputs, batch_labels)
            batch_loss.backward()
            optimizer.step()
            epoch_loss += batch_loss.item() * batch_inputs.size(0)
            if batch_index % 5 == 0:
                print("EPOCH {} BATCH {}: training batch loss: {}".format(epoch+1, batch_index+1, batch_loss.item()))

            if batch_index % 10 == 0:
                phase = 'val'
                val_preds = np.zeros((dataset_sizes['val'], 1))
                val_loss = 0.0
                with torch.no_grad():
                    model.eval()
                    for val_batch_index, (val_batch_inputs, val_batch_labels) in enumerate(dataloaders[phase]):

                        val_batch_inputs = val_batch_inputs.cuda()
                        val_batch_labels = val_batch_labels.cuda()

                        val_outputs = model(val_batch_inputs).detach()
                        _, val_batch_preds = torch.max(val_outputs, 1)
                        val_batch_loss = criterion(val_outputs, val_batch_labels)
                        val_preds[val_batch_index * batch_size: (val_batch_index+1) * batch_size] = val_batch_preds.cpu().view(-1, 1).numpy()
                        val_loss += val_batch_loss.item() * val_batch_inputs.size(0)

                    val_score = f1_score(y_val, val_preds, average='micro')
                    print()
                    print(">>>>>>  EPOCH {} BATCH {}: validation score {}".format(epoch+1, batch_index+1, val_score))
                    print()
                    if val_score > best_f1:
                        best_f1 = val_score
                        best_model_weights = copy.deepcopy(model.state_dict())
                        torch.save(model.state_dict(), '../{}/best_model_{}_{}.pt'.format(PATH, epoch+1, batch_index+1))

                phase = 'train'
                model.train()

        epoch_loss = epoch_loss / dataset_sizes['train']
        print("EPOCH {}: EPOCH_LOSS: {}".format(epoch+1, epoch_loss))
    end = time.time()
    elapsed_time = end - start
    print("Training COMPLETED: {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("BEST VALIDATION F1: {:4f}".format(best_f1))

    model.load_state_dict(best_model_weights)
    return model


# model_res.to(device)
# optimizer = optim.Adam(model_res.parameters(), lr=0.000001)
# criterion = nn.CrossEntropyLoss()
# model_res = train_model(model=model_res, dataloaders=dataloaders, dataset_sizes=dataset_sizes, criterion=criterion, optimizer=optimizer, device=device, epochs=100, PATH='model/ten_crop/fine_tune')



In [7]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('../model/ten_crop/tune2/best_model_8_91.pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [6]:
model_res.to(device)
optimizer = optim.Adam(model_res.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

model_res = train_model(model=model_res, dataloaders=dataladers, dataset_sizes=dataset_sizes, criterion=criterion, optimizer=optimizer, device=device, epochs=10, 
        PATH='model/ten_crop/lr_000001')

  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH 1 / 10: 
----------
EPOCH 1 BATCH 1: training batch loss: 0.08228839188814163

>>>>>>  EPOCH 1 BATCH 1: validation score 0.8928928928928928

EPOCH 1 BATCH 6: training batch loss: 0.05276988819241524
EPOCH 1 BATCH 11: training batch loss: 0.046658001840114594

>>>>>>  EPOCH 1 BATCH 11: validation score 0.9054054054054054

EPOCH 1 BATCH 16: training batch loss: 0.041372254490852356
EPOCH 1 BATCH 21: training batch loss: 0.06828468292951584

>>>>>>  EPOCH 1 BATCH 21: validation score 0.9074074074074074

EPOCH 1 BATCH 26: training batch loss: 0.03543846309185028
EPOCH 1 BATCH 31: training batch loss: 0.07012776285409927

>>>>>>  EPOCH 1 BATCH 31: validation score 0.9054054054054054

EPOCH 1 BATCH 36: training batch loss: 0.052548304200172424
EPOCH 1 BATCH 41: training batch loss: 0.04351343214511871

>>>>>>  EPOCH 1 BATCH 41: validation score 0.9119119119119119

EPOCH 1 BATCH 46: training batch loss: 0.04557860642671585
EPOCH 1 BATCH 51: training batch loss: 0.021260283887386322

>>>

 10%|█         | 1/10 [13:51<2:04:43, 831.55s/it]

EPOCH 1: EPOCH_LOSS: 0.0478987575241097
EPOCH 2 / 10: 
----------
EPOCH 2 BATCH 1: training batch loss: 0.04848185181617737

>>>>>>  EPOCH 2 BATCH 1: validation score 0.9134134134134134

EPOCH 2 BATCH 6: training batch loss: 0.06022125482559204
EPOCH 2 BATCH 11: training batch loss: 0.030838459730148315

>>>>>>  EPOCH 2 BATCH 11: validation score 0.9164164164164165

EPOCH 2 BATCH 16: training batch loss: 0.04105565696954727
EPOCH 2 BATCH 21: training batch loss: 0.04485087841749191

>>>>>>  EPOCH 2 BATCH 21: validation score 0.913913913913914

EPOCH 2 BATCH 26: training batch loss: 0.04826447367668152
EPOCH 2 BATCH 31: training batch loss: 0.028922419995069504

>>>>>>  EPOCH 2 BATCH 31: validation score 0.9119119119119119

EPOCH 2 BATCH 36: training batch loss: 0.013672314584255219
EPOCH 2 BATCH 41: training batch loss: 0.022655636072158813

>>>>>>  EPOCH 2 BATCH 41: validation score 0.9134134134134134

EPOCH 2 BATCH 46: training batch loss: 0.02171725034713745
EPOCH 2 BATCH 51: traini

 20%|██        | 2/10 [27:42<1:50:49, 831.24s/it]

EPOCH 2: EPOCH_LOSS: 0.030731403333199275
EPOCH 3 / 10: 
----------
EPOCH 3 BATCH 1: training batch loss: 0.025953002274036407

>>>>>>  EPOCH 3 BATCH 1: validation score 0.9144144144144143

EPOCH 3 BATCH 6: training batch loss: 0.017689935863018036
EPOCH 3 BATCH 11: training batch loss: 0.011263072490692139

>>>>>>  EPOCH 3 BATCH 11: validation score 0.915915915915916

EPOCH 3 BATCH 16: training batch loss: 0.034100234508514404
EPOCH 3 BATCH 21: training batch loss: 0.01744462549686432

>>>>>>  EPOCH 3 BATCH 21: validation score 0.915915915915916

EPOCH 3 BATCH 26: training batch loss: 0.03080083429813385
EPOCH 3 BATCH 31: training batch loss: 0.03105621039867401

>>>>>>  EPOCH 3 BATCH 31: validation score 0.9169169169169169

EPOCH 3 BATCH 36: training batch loss: 0.029327332973480225
EPOCH 3 BATCH 41: training batch loss: 0.012172400951385498

>>>>>>  EPOCH 3 BATCH 41: validation score 0.914914914914915

EPOCH 3 BATCH 46: training batch loss: 0.02569584548473358
EPOCH 3 BATCH 51: trai

 30%|███       | 3/10 [41:32<1:36:56, 830.96s/it]

EPOCH 3: EPOCH_LOSS: 0.02337380858684505
EPOCH 4 / 10: 
----------
EPOCH 4 BATCH 1: training batch loss: 0.024378828704357147

>>>>>>  EPOCH 4 BATCH 1: validation score 0.918918918918919

EPOCH 4 BATCH 6: training batch loss: 0.020179763436317444
EPOCH 4 BATCH 11: training batch loss: 0.019267529249191284

>>>>>>  EPOCH 4 BATCH 11: validation score 0.9169169169169169

EPOCH 4 BATCH 16: training batch loss: 0.019304849207401276
EPOCH 4 BATCH 21: training batch loss: 0.015996553003787994

>>>>>>  EPOCH 4 BATCH 21: validation score 0.9164164164164165

EPOCH 4 BATCH 26: training batch loss: 0.014765970408916473
EPOCH 4 BATCH 31: training batch loss: 0.020187579095363617

>>>>>>  EPOCH 4 BATCH 31: validation score 0.9179179179179179

EPOCH 4 BATCH 36: training batch loss: 0.024215087294578552
EPOCH 4 BATCH 41: training batch loss: 0.024176180362701416

>>>>>>  EPOCH 4 BATCH 41: validation score 0.918918918918919

EPOCH 4 BATCH 46: training batch loss: 0.02856183797121048
EPOCH 4 BATCH 51: t

KeyboardInterrupt: 

In [11]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('../model/ten_crop/lr_00001/best_model_3_111.pt'))
model_res.to(device)
optimizer = optim.Adam(model_res.parameters(), lr=0.000001)
criterion = nn.CrossEntropyLoss()

model_res = train_model(model=model_res, dataloaders=dataloaders, dataset_sizes=dataset_sizes, criterion=criterion, optimizer=optimizer, device=device, epochs=10, 
        PATH='model/ten_crop/lr_000001')




  0%|          | 0/10 [00:00<?, ?it/s]

EPOCH 1 / 10: 
----------
EPOCH 1 BATCH 1: training batch loss: 0.03172450512647629

>>>>>>  EPOCH 1 BATCH 1: validation score 0.9194194194194194

EPOCH 1 BATCH 6: training batch loss: 0.014794662594795227
EPOCH 1 BATCH 11: training batch loss: 0.009986512362957

>>>>>>  EPOCH 1 BATCH 11: validation score 0.91991991991992

EPOCH 1 BATCH 16: training batch loss: 0.04364961385726929
EPOCH 1 BATCH 21: training batch loss: 0.011511467397212982

>>>>>>  EPOCH 1 BATCH 21: validation score 0.9194194194194194

EPOCH 1 BATCH 26: training batch loss: 0.08644595742225647
EPOCH 1 BATCH 31: training batch loss: 0.015699051320552826

>>>>>>  EPOCH 1 BATCH 31: validation score 0.9194194194194194

EPOCH 1 BATCH 36: training batch loss: 0.01607094705104828
EPOCH 1 BATCH 41: training batch loss: 0.036269813776016235

>>>>>>  EPOCH 1 BATCH 41: validation score 0.9164164164164165

EPOCH 1 BATCH 46: training batch loss: 0.03038366138935089
EPOCH 1 BATCH 51: training batch loss: 0.021399974822998047

>>>>>>




 10%|█         | 1/10 [13:47<2:04:03, 827.02s/it]

EPOCH 1: EPOCH_LOSS: 0.02057485068934756
EPOCH 2 / 10: 
----------
EPOCH 2 BATCH 1: training batch loss: 0.02636047452688217

>>>>>>  EPOCH 2 BATCH 1: validation score 0.9164164164164165

EPOCH 2 BATCH 6: training batch loss: 0.01388617604970932
EPOCH 2 BATCH 11: training batch loss: 0.01684536784887314

>>>>>>  EPOCH 2 BATCH 11: validation score 0.918918918918919

EPOCH 2 BATCH 16: training batch loss: 0.022331316024065018
EPOCH 2 BATCH 21: training batch loss: 0.028845056891441345

>>>>>>  EPOCH 2 BATCH 21: validation score 0.91991991991992

EPOCH 2 BATCH 26: training batch loss: 0.011186376214027405
EPOCH 2 BATCH 31: training batch loss: 0.023941881954669952

>>>>>>  EPOCH 2 BATCH 31: validation score 0.9194194194194194

EPOCH 2 BATCH 36: training batch loss: 0.015141338109970093
EPOCH 2 BATCH 41: training batch loss: 0.008480347692966461

>>>>>>  EPOCH 2 BATCH 41: validation score 0.91991991991992

EPOCH 2 BATCH 46: training batch loss: 0.015148505568504333
EPOCH 2 BATCH 51: traini




 20%|██        | 2/10 [27:32<1:50:13, 826.66s/it]

...




 40%|████      | 4/10 [55:13<1:22:52, 828.68s/it]

EPOCH 4: EPOCH_LOSS: 0.017857775670122932
EPOCH 5 / 10: 
----------
EPOCH 5 BATCH 1: training batch loss: 0.010121278464794159

>>>>>>  EPOCH 5 BATCH 1: validation score 0.9179179179179179

EPOCH 5 BATCH 6: training batch loss: 0.013596661388874054
EPOCH 5 BATCH 11: training batch loss: 0.012511976063251495

>>>>>>  EPOCH 5 BATCH 11: validation score 0.91991991991992

EPOCH 5 BATCH 16: training batch loss: 0.011072732508182526
EPOCH 5 BATCH 21: training batch loss: 0.010629832744598389

>>>>>>  EPOCH 5 BATCH 21: validation score 0.9194194194194194

EPOCH 5 BATCH 26: training batch loss: 0.014546878635883331
EPOCH 5 BATCH 31: training batch loss: 0.014782153069972992

>>>>>>  EPOCH 5 BATCH 31: validation score 0.915915915915916

EPOCH 5 BATCH 36: training batch loss: 0.016953110694885254
EPOCH 5 BATCH 41: training batch loss: 0.013016700744628906

>>>>>>  EPOCH 5 BATCH 41: validation score 0.9184184184184183

EPOCH 5 BATCH 46: training batch loss: 0.013035997748374939
EPOCH 5 BATCH 51: 




 50%|█████     | 5/10 [1:09:02<1:09:03, 828.68s/it]

EPOCH 5: EPOCH_LOSS: 0.015716561685885873
EPOCH 6 / 10: 
----------
EPOCH 6 BATCH 1: training batch loss: 0.009658291935920715

>>>>>>  EPOCH 6 BATCH 1: validation score 0.9179179179179179



KeyboardInterrupt: 

In [19]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('../model/ten_crop/lr_000001/best_model_3_121.pt'))
model_res.to(device)
optimizer = optim.Adam(model_res.parameters(), lr=1e-7)
criterion = nn.CrossEntropyLoss()

model_res = train_model(model=model_res, dataloaders=dataloaders, dataset_sizes=dataset_sizes, criterion=criterion, optimizer=optimizer, device=device, epochs=4, 
        PATH='model/ten_crop/lr_1e-7')





  0%|          | 0/4 [00:00<?, ?it/s]

EPOCH 1 / 4: 
----------
EPOCH 1 BATCH 1: training batch loss: 0.013814657926559448

>>>>>>  EPOCH 1 BATCH 1: validation score 0.9229229229229229

EPOCH 1 BATCH 6: training batch loss: 0.019834503531455994
EPOCH 1 BATCH 11: training batch loss: 0.01316262036561966

>>>>>>  EPOCH 1 BATCH 11: validation score 0.9209209209209209

EPOCH 1 BATCH 16: training batch loss: 0.011571168899536133
EPOCH 1 BATCH 21: training batch loss: 0.03169119730591774

>>>>>>  EPOCH 1 BATCH 21: validation score 0.9209209209209209

EPOCH 1 BATCH 26: training batch loss: 0.029147803783416748
EPOCH 1 BATCH 31: training batch loss: 0.010605335235595703

>>>>>>  EPOCH 1 BATCH 31: validation score 0.918918918918919

EPOCH 1 BATCH 36: training batch loss: 0.017316393554210663
EPOCH 1 BATCH 41: training batch loss: 0.018457621335983276

>>>>>>  EPOCH 1 BATCH 41: validation score 0.91991991991992

EPOCH 1 BATCH 46: training batch loss: 0.015717744827270508
EPOCH 1 BATCH 51: training batch loss: 0.033132731914520264

>>





 25%|██▌       | 1/4 [13:46<41:20, 826.92s/it]

EPOCH 1: EPOCH_LOSS: 0.01748569445168083
EPOCH 2 / 4: 
----------
EPOCH 2 BATCH 1: training batch loss: 0.010323449969291687


KeyboardInterrupt: 

In [34]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('../model/ten_crop/lr_000001/best_model_3_121.pt'))

model_res.to(device)

df_test = pd.read_csv('../data/test.csv')
X_test = df_test['img_file'].values

test_dataset = TestImages(X_test, mode='test', transforms=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model_res.eval()
test_preds = []

with torch.no_grad():
    for i, images in enumerate(test_loader):
        images = images.cuda()
    
        preds = model_res(images).detach()
        test_preds.append(preds.cpu().numpy())

In [35]:
outputs = []
for _ in test_preds:
    predicted_class_indices=np.argmax(_, axis=1).tolist()
    outputs.append(predicted_class_indices)

result = np.concatenate(outputs)

In [37]:
submission = pd.read_csv('../data/sample_submission.csv')
submission["class"] = result
submission["class"].replace(0, 196, inplace=True) # 196에서 0으로 수정했던걸 다시 되돌려준다 
submission.to_csv("../data/submission.csv", index=False)
submission.head()

,img_file,class
0,test_00001.jpg,124
1,test_00002.jpg,98
2,test_00003.jpg,157
3,test_00004.jpg,94
4,test_00005.jpg,18
